<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/CreateGSPS_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Аннотация

* при формировании в папке ТЗ/ДОГОВОРЫ документа с данными, автоматически формируется SS, состоящая из следующих листов: FINANCE, PLAN, COST, CHART'S

    * FINANCE это клон Приложения 1 к соответствующему договору + календарь проекта. Данные отображаемые на листе: прогнозный расчет КС на фиксированную дату (TODO PLAN_RADAR, FAKT), фактическая КС, формирование плана финансирования/факта финансирования
    (TODO FINANCE_FAKT !!!)
    * PLAN формируется в полуавтоматическом режиме => шаблон формируется скриптом в разрезе локальных зон объекта, с учетом выработки по базе ППР => инженер корректирует захватки в МОДЕЛИ (TODO!!!, как внести сразу и даты??? - VIEWER). Фактические данные попадают ТОЛЬКО через PLAN_RADAR, в "формах" стоит триггер START/ВЫПОЛНЕНО/ПРИНЯТО/FINISH (TODO формы и сonnect PLAN_RADAR)
    * COST шаблон формируется на основании внутнней БД и обновляется в разрезе стоимости материалов на основании фактических данных (накладные). Факт поставки подтверждается: загрузка накладных в папку АСС/модлем СКЛАД=>поставка
    * CHART'S: план недели, график недели, платежи недели, поставка недели, платежи недели

**ЦЕЛИ:**
 

1.   Подтвердить достаточность ресурсов для выполнения работ на соответствующем временном плече
2.   Оценка риска недостаточности финансирования на сроках более месяца
3.   Оценка риска достаточности материалов на строительной площадке и общее выполнение графика поставки материалов



In [1]:
from google.colab import auth
auth.authenticate_user()

import seaborn as sns
import pandas as pd
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime
from datetime import timedelta

# worksheet = gc.open('План работ Договор Адамант').sheet1
worksheet = gc.open('План работ Договор Адамант')

# get_all_values gives a list of rows.
# rows = worksheet.get_all_values()
# print(rows)

# # Convert to a DataFrame and render.
# pd.DataFrame.from_records(rows)

In [2]:
idx = pd.IndexSlice

* TODO данные финансового плана импортируются из АСС => папка соответсвующего договора => Приложение 1 (рапределение договорной стоимости) формируется НИКОЛИРЕС на этапе формирования ТЗ 

\begin{align} 
\mathbf{TO=>ENTER:  } passwordBudget, BD_{pricing},  ITEM'S_{model}
\end{align}


\begin{align} 
\mathbf{TO=>EXIT:  } COST
\end{align}



\begin{align} 
\mathbf{COST} &= \sum_{i=1}^N \mathbf{pricing_i} \cdot \mathbf{valueModel_i}\end{align}

\begin{align} 
\mathbf{N} - {1...N}\end{align}

\begin{align} 
\mathbf{pricing} &= passwordBudget + password_{BD} + name + workers + mashins + BD_{materials}
\end{align}

\begin{align} 
\mathbf{valueModel} &= password_{model} + name + area/value/item
\end{align}

\begin{align} 
\mathbf{password_{COST}} &= password_{model} + password_{BD}
\end{align}


## Формирование листа FINANCE
=> АСС/ДОГОВОР/Приложение 2
Структура

1. \begin{align} \mathbf{password_{COST} => text} \end{align}

2. \begin{align} \mathbf{NAME => text} \end{align}

3. \begin{align} \mathbf{START => text} \end{align} 

4. \begin{align} \mathbf{FINISH => text} \end{align} 

5. \begin{align} \mathbf{pricing_{item} => pd.to_numeric} \end{align} 

6. \begin{align} \mathbf{valueModel => pd.to_numeric} \end{align} 

7. \begin{align} \mathbf{TOTAL => pd.to_numeric} \end{align} 
 
8. \begin{align} \frac{date}{PLAN} => text\end{align} 

9. \begin{align} \frac{date}{FAKT} => text\end{align} 

* TODO логика раздела будет переписана после появления базы расценок

In [3]:
fin = worksheet.worksheet("FINANCE PLAN (копия)")
fin = fin.get_all_values()
fin = np.array(fin)
fin =  pd.DataFrame(fin[5:, :7])
fin.drop(5, axis=1, inplace=True)

In [4]:
new_fin = fin.copy()
new_fin
new_fin.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 4:'pricingItem', 6:'valueModel'}, 
            inplace = True)
new_fin = new_fin.set_index('passwordCOST')
new_fin = new_fin[new_fin.index > '']

Приводим строковые значения к числовым 

In [5]:
new_fin['pricingItem'] = new_fin['pricingItem'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
new_fin['valueModel'] = new_fin['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
new_fin['TOTAL'] = new_fin['pricingItem'] * new_fin['valueModel']

Приводим значения дат в формат date

In [6]:
new_fin = new_fin[new_fin['START'] > '']
new_fin = new_fin[new_fin['FINISH'] > '']

In [7]:
new_fin['START'] = pd.to_datetime(pd.Series(new_fin['START']).apply(lambda x: str(x)))
new_fin['FINISH'] = pd.to_datetime(pd.Series(new_fin['FINISH']).apply(lambda x: str(x)))

Завершаем предобработку таблицы - заменяем все значения Nan на "0"

In [8]:
# на этом этапе формируем лист PLAN
new_fin = new_fin.fillna(0)

In [9]:
new_fin['TOTAL'] = new_fin['pricingItem'] * new_fin['valueModel']

In [ ]:
new_fin

## Формирование листа PLAN

PLAN = FINANCE\['TOTAL']

* TODO логика раздела будет переписана после появления базы расценок
* ❌нельзя добавлять новые виды работ
* ❗ детализация на захватки происходит либо в модели, либо на чертеже => в таблицу данные заносятся самостоятельно
* ❗ таблица оповещает о выходе за границы временного промежутка 
* ❗ подсчет рабочих по специальностям и машмех еще не заложен в логику

* created function's: cumulativePlan, cumulativeFakt

In [11]:
graf = worksheet.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf = pd.DataFrame(graf[1:, :])

In [12]:
new_graf = graf.copy()
new_graf = new_graf.loc[idx[2:],idx[:6]]
new_graf.drop([4,5], axis=1, inplace=True)

In [13]:
new_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
new_graf = new_graf[new_graf['passwordCOST'] > '']
new_graf = new_graf[new_graf['START'] > '']
new_graf = new_graf[new_graf['FINISH'] > '']
new_graf = new_graf.set_index('passwordCOST')
new_graf['valueModel'] = new_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
new_graf = new_graf.fillna(0)

In [ ]:
new_graf

❗: ниже приведенный код исполняется временно, до момента автоматического формирования плана
❗

In [15]:
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])

In [ ]:
graf_name

In [17]:
graf.drop(graf.index[1:10], axis=1, inplace=True)

In [18]:
newColumnsIndex = [pd.to_datetime(pd.Series(graf.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf.iloc[1][1:].values]

In [20]:
graf.rename(columns={0: 'passwordCOST'}, inplace=True)

In [22]:
graf = graf.set_index("passwordCOST")
graf = pd.DataFrame(graf.values, index=graf.index, columns=newColumnsIndex)
graf.drop(graf.index[:2], axis=0, inplace=True)
for i in graf.columns:
    graf[i] = graf[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))

In [ ]:
graf

In [ ]:
graf.fillna(0, inplace=True)
graf.sort_index(axis=0, level=1, ascending=True)
graf = graf[graf.index > '']
graf

In [21]:
fin.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 4:'pricingItem', 6:'valueModel'}, 
            inplace = True)

In [22]:
fin = fin.set_index('passwordCOST')
fin = fin[fin.index > '']

In [23]:
fin['pricingItem'] = fin['pricingItem'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
fin['VALUE CONTRACT'] = fin['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))

In [24]:
fin['START'] = pd.to_datetime(fin['START'])   
fin['FINISH']= pd.to_datetime(fin['FINISH'])   

In [ ]:
fin = fin.fillna(0)
fin

❌ см. предидущий комментарий

## Формируем ЕДИНЫЙ DATASET

### Формируем график проекта

Индексы подзадач:
* level1 DATE => pd.to_datetime()
* level2 plan/fakt => str()
* level3 :
                    " " => None
                    "KS-2 # {номер КС-2}" => str()
                    "FIN_PLAN # {номер плана финансирования}" => str()
                    "SMR" => str()
                    "MATERIALS" => str()
                    "WORKER'S" => str()
                    "UNIT'S" => str()
                    "PD" => str()
                    "RD" => str()
                    "KT1" => str()
                    "KT2" => str()
                    "KT3" => str()




In [26]:
# формируем level1
start = '02.01.2022'
finish = '02.02.2023'
dateIndex = pd.date_range(start=start, end=finish)
dataIndex = dateIndex.map(lambda z: pd.to_datetime(z))
# dataIndex

In [27]:
# добавляем level2 и шаблон для level3
multiindex = pd.MultiIndex.from_product([dateIndex,['plan','fakt'],[" "]], names=["DATE", "PLAN/FAKT"," "])

### TOTAL_DATA: 

* index => Finance.index
* dataset => multiindex

In [28]:
totalData = pd.DataFrame(index=new_fin.index, columns=multiindex)
totalData = totalData.fillna(0)

In [29]:
totalData

DATE                     2022-02-01      2022-02-02      2022-02-03       \
PLAN/FAKT                      plan fakt       plan fakt       plan fakt   
                                                                           
passwordCOST                                                               
VZO_POS1_POD_MOB_.                0    0          0    0          0    0   
VZO_POS1_POD_VRD_DP17.01          0    0          0    0          0    0   
VZO_POS1_POD_VRS_ES0.01           0    0          0    0          0    0   
VZO_POS1_POD_VRS_VK0.01           0    0          0    0          0    0   
VZO_POS1_POD_VRD_DP17.02          0    0          0    0          0    0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                        0    0          0    0          0    0   
BOT_GP                            0    0          0    0          0    0   
_EXP___.                          0    0          0    0          0    0   
_ZOS___.                          0    0          0    0          0    0   
_RNV___.                          0    0          0    0          0    0   

DATE                     2022-02-04      2022-02-05       ... 2023-01-29       \
PLAN/FAKT                      plan fakt       plan fakt  ...       plan fakt   
                                                          ...                   
passwordCOST                                              ...                   
VZO_POS1_POD_MOB_.                0    0          0    0  ...          0    0   
VZO_POS1_POD_VRD_DP17.01          0    0          0    0  ...          0    0   
VZO_POS1_POD_VRS_ES0.01           0    0          0    0  ...          0    0   
VZO_POS1_POD_VRS_VK0.01           0    0          0    0  ...          0    0   
VZO_POS1_POD_VRD_DP17.02          0    0          0    0  ...          0    0   
...                             ...  ...        ...  ...  ...        ...  ...   
BOT_GP___.                        0    0          0    0  ...          0    0   
BOT_GP                            0    0          0    0  ...          0    0   
_EXP___.                          0    0          0    0  ...          0    0   
_ZOS___.                          0    0          0    0  ...          0    0   
_RNV___.                          0    0          0    0  ...          0    0   

DATE                     2023-01-30      2023-01-31      2023-02-01       \
PLAN/FAKT                      plan fakt       plan fakt       plan fakt   
                                                                           
passwordCOST                                                               
VZO_POS1_POD_MOB_.                0    0          0    0          0    0   
VZO_POS1_POD_VRD_DP17.01          0    0          0    0          0    0   
VZO_POS1_POD_VRS_ES0.01           0    0          0    0          0    0   
VZO_POS1_POD_VRS_VK0.01           0    0          0    0          0    0   
VZO_POS1_POD_VRD_DP17.02          0    0          0    0          0    0   
...                             ...  ...        ...  ...        ...  ...   
BOT_GP___.                        0    0          0    0          0    0   
BOT_GP                            0    0          0    0          0    0   
_EXP___.                          0    0          0    0          0    0   
_ZOS___.                          0    0          0    0          0    0   
_RNV___.                          0    0          0    0          0    0   

DATE                     2023-02-02       
PLAN/FAKT                      plan fakt  
                                          
passwordCOST                              
VZO_POS1_POD_MOB_.                0    0  
VZO_POS1_POD_VRD_DP17.01          0    0  
VZO_POS1_POD_VRS_ES0.01           0    0  
VZO_POS1_POD_VRS_VK0.01           0    0  
VZO_POS1_POD_VRD_DP17.02          0    0  
...                             ...  ...  
BOT_GP___.                        0    0  
BOT_GP                       

#### Расчет SMR plan/fakt

* to ENTER: 
            START => str()
            FINISH => str()

##### Расчет ПЛАНА КС-2

❗ применяется текущая модель формирования графика - graf, fin

In [70]:
# start and finish input in filter backend
start = '30.12.2022'
start = pd.to_datetime(start)
finish = '16.01.2023'
finish = pd.to_datetime(finish)
start, finish

(Timestamp('2022-12-30 00:00:00'), Timestamp('2023-01-16 00:00:00'))

In [71]:
# number KS-2 input in backend
numberKS_2 = '2'
numberKS_2 = pd.to_numeric(numberKS_2)
nameKS_2 = 'КС-2 №{}'.format(numberKS_2)

In [72]:
plan_KS2 = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values
plan_KS2 = pd.DataFrame(plan_KS2,index=graf.index, columns=pd.MultiIndex.from_tuples([(finish, 'plan', nameKS_2)]))

In [73]:
plan_KS2 = plan_KS2.loc[list(set(plan_KS2.index) & set(fin.index))]
plan_KS2 = plan_KS2.join(fin['pricingItem'])
plan_KS2['total'] = plan_KS2.iloc[:, 0] * plan_KS2.iloc[:, 1]
plan_KS2 = pd.DataFrame(plan_KS2['total'],index=plan_KS2.index, columns=pd.MultiIndex.from_tuples([(finish, 'plan', nameKS_2)]))

In [34]:
totalData = totalData.join(plan_KS2)
totalData = totalData.sort_index(axis=1, level=0)

❌ см. предидущий комментарий

##### Расчет ФАКТ КС-2 

Добавляем из фактической КС на АСС

#### Расчет плана финансирования

In [35]:
# данные вводит пользователь
start = '30.12.2022'
start = pd.to_datetime(start)
finish = '16.01.2023'
finish = pd.to_datetime(finish)
start, finish

(Timestamp('2022-12-30 00:00:00'), Timestamp('2023-01-16 00:00:00'))

In [36]:
# данные вводит пользователь
rate_plan = 7
numberPlan = 1
# namePlan = 'FinPlan №{}'.format(numberPlan)

In [37]:
listStart = pd.date_range(start=start, end=finish, freq='{}D'.format(rate_plan))
listFinish = pd.date_range(start=(start - timedelta(days=1)), end=finish, freq='{}D'.format(rate_plan))

In [38]:
datePlan = list(zip(listStart,listFinish))

In [39]:
indx = 1
totalPay = pd.DataFrame(fin['pricingItem'].values, index=fin.index, columns=['pricingItem'])
total =[]

In [40]:
for i in datePlan:
    start = pd.Timestamp(i[0]).date()
    finish = pd.Timestamp(i[1]).date()
    namePay = 'PayPlan #{} pay #{}'.format(numberPlan, indx)

    plan_PAY = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values
    # plan_PAY = pd.DataFrame(plan_PAY,index=graf.index, columns=pd.MultiIndex.from_tuples([(finish, 'plan', 'value')]))
    plan_PAY = pd.DataFrame(plan_PAY,index=graf.index)
    plan_PAY = plan_PAY.loc[list(set(totalPay.index) & set(plan_PAY.index))]
    plan_PAY = plan_PAY.join(totalPay['pricingItem'])
    plan_PAY[(finish, 'plan', namePay)] = plan_PAY['pricingItem'] * plan_PAY.iloc[:, 0]
    # plan_PAY[(finish, 'plan', namePay)] = plan_PAY.iloc[:, indx-1] * totalPay['pricingItem']
    total.append(plan_PAY.iloc[:, idx[-1]])

    indx += 1

In [41]:
for i in total:
    totalPay = totalPay.join(i)

In [42]:
totalPay = totalPay.iloc[:, idx[1:]]

In [43]:
totalData = totalData.join(totalPay)
totalData = totalData.sort_index(axis=1, level=0)

In [ ]:
totalData

In [45]:
# worksheet.worksheet('PLAN_WEEK').update_cells()

In [ ]:
# worksheet.worksheet('PLAN_WEEK').append_rows([planFakt.columns.values.tolist()]+planFakt.values.tolist())

In [ ]:
# данный элемент добавляется после каждого участка добавления данных
# multiindex = multiindex.append(pd.MultiIndex.from_tuples([(pd.to_datetime('02.02.2022'), 'plan', 'Plan #{} {}'.format(1,3))],names=("DATE", "PLAN/FAKT"," ")))
# multiindex.sortlevel(level=0)

In [ ]:
dir(worksheet.worksheet('PLAN_WEEK'))

# обновление плана недели

In [159]:
start_week = '30.12.2022'
start_week = pd.to_datetime(start_week)
finish_week = '16.01.2023'
finish_week = pd.to_datetime(finish_week)
# ----------------------------------------------------
start_next_week = '17.01.2023'
start_next_week = pd.to_datetime(start_next_week)
finish_next_week = '20.01.2023'
finish__next_week = pd.to_datetime(finish_next_week)
# -----------------------------------------------------
start_project = '01.02.2022'
start_project = pd.to_datetime(start_project)
finish_week = '16.01.2023'
finish_week = pd.to_datetime(finish_week)

In [160]:
graf_week_plan = graf.loc[:, idx[start_week:finish_week,'план']].sum(axis=1).values
graf_week_plan = pd.DataFrame(graf_week_plan,index=graf.index, columns=['PLAN_WEEK'])
graf_week_fakt = graf.loc[:, idx[start_week:finish_week,'факт']].sum(axis=1).values
graf_week_fakt = pd.DataFrame(graf_week_fakt,index=graf.index, columns=['FAKT_WEEK'])

In [161]:
graf_next_week_plan = graf.loc[:, idx[start_next_week:finish_next_week,'план']].sum(axis=1).values
graf_next_week_plan = pd.DataFrame(graf_next_week_plan,index=graf.index, columns=['PLAN_NEXT_WEEK'])
graf_next_week_fakt = graf.loc[:, idx[start_next_week:finish_next_week,'факт']].sum(axis=1).values
graf_next_week_fakt = pd.DataFrame(graf_next_week_fakt,index=graf.index, columns=['FAKT_NEXT_WEEK'])

In [162]:
total_plan = graf.loc[:, idx[start_project:finish_week,'план']].sum(axis=1).values
total_plan = pd.DataFrame(total_plan,index=graf.index, columns=['TOTAL_PLAN'])
total_fakt = graf.loc[:, idx[start_project:finish_week,'факт']].sum(axis=1).values
total_fakt = pd.DataFrame(total_fakt,index=graf.index, columns=['TOTAL_FAKT'])

In [163]:
new_graf_week = new_graf.join([total_plan,total_fakt,graf_week_plan,graf_week_fakt,graf_next_week_plan,graf_next_week_fakt])

In [164]:
new_graf_week = new_graf_week[new_graf_week['PLAN_WEEK']>0]

In [165]:
new_graf_week = new_graf_week[['NAME', 'TOTAL_PLAN','TOTAL_FAKT','PLAN_WEEK','FAKT_WEEK','PLAN_NEXT_WEEK','FAKT_NEXT_WEEK']]

In [166]:
new_graf_week['%TOTAl_WEEK'] =( new_graf_week['FAKT_WEEK'] / new_graf_week['PLAN_WEEK']).mul(100).round(2).astype(str).add(' %')

In [167]:
new_graf_week['%TOTAl'] =( new_graf_week['TOTAL_FAKT'] / new_graf_week['TOTAL_PLAN']).mul(100).round(2).astype(str).add(' %')

In [168]:
new_graf_week.rename(columns={'NAME':f'WORK on {pd.to_datetime(start_week).date()}  to {pd.to_datetime(finish_week).date()}'}, inplace=True)

In [ ]:
worksheet.worksheet('PLAN_WEEK').clear()

In [ ]:
worksheet.worksheet('PLAN_WEEK').append_rows([new_graf_week.columns.values.tolist()]+new_graf_week.values.tolist())